# Basic web scraping and analysis
This script will extract the table information from a wikipedia link and house the information in a data frame for later processing.

In [116]:
import numpy as np
import pandas as pd
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Umesh Jonnalagadda\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.4                |   py37hc8dfbb8_2         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The fol



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda




In [102]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    
df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])

### Part 1: Web data extraction to data frame

In [103]:
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')
nulCase = 'Not assigned'
skipRow = False
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    postCode = data[0].text[0:-1]
    borough = data[1].text[0:-1]
    neighbourhood = data[2].text[0:-1]

    if borough == nulCase:
        skipRow = True

    if neighbourhood == nulCase and borough != nulCase:
        neighborhood = borough

    newRow = {'Postal Code' : postCode, 'Borough' : borough, 'Neighbourhood' : neighbourhood}
    
    if skipRow == False:
        df = df.append(newRow, ignore_index = True)
    skipRow = False

In [106]:
df.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [108]:
df.shape

(103, 3)

### Part 2: Merging Neighborhood data with location data

In [114]:
locationDF = pd.read_csv(r'./Geospatial_Coordinates.csv')
locationDF.head()

ModuleNotFoundError: No module named 'geopy'

Postal code is the 

In [112]:
mergedDF = pd.merge(left = df, right = locationDF, left_on = 'Postal Code', right_on = 'Postal Code')
mergedDF.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Part 3: Location Clustering

In [ ]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
    
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)